# Let's try something different than Gradient Boosting...

## Table of Contents
* [Target Exploration](#1)
* [Numerical Features](#2)
* [Categorical Features](#3)
* [Target vs Features](#4)
* [Build GLM Model](#5)
* [Build Random Forest Model](#6)
* [Predict on Test Set and prepare submissions](#6)

In [ ]:
# packages

# standard
import numpy as np
import pandas as pd
import time

# plots
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.graphics.mosaicplot import mosaic

# machine learning tools
import h2o
from h2o.estimators import H2OGeneralizedLinearEstimator, H2ORandomForestEstimator

In [ ]:
# load data + first glance
df_train = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
df_test = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')
df_sub = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')

# first glance (training data)
df_train.head()

In [ ]:
# dimensions
df_train.shape

In [ ]:
df_train.info()

#### We are lucky, no missing values!

<a id='1'></a>
# Target Exploration

### This time we have a categorical (binary) target!

In [ ]:
# basic stats
print(df_train.target.value_counts())
df_train.target.value_counts().plot(kind='bar')
plt.grid()
plt.show()

<a id='2'></a>
# Numerical Features

In [ ]:
features_num = ['cont0', 'cont1', 'cont2', 'cont3', 
                'cont4', 'cont5', 'cont6', 'cont7',
                'cont8', 'cont9', 'cont10']

In [ ]:
# plot distribution of numerical features
for f in features_num:
    plt.figure(figsize=(8,4))
    df_train[f].plot(kind='hist', bins=100)
    plt.title(f)
    plt.grid()
    plt.show()

## Feature Correlations

In [ ]:
corr_pearson = df_train[features_num].corr(method='pearson')
corr_spearman = df_train[features_num].corr(method='spearman')

fig = plt.figure(figsize = (10,8))
sns.heatmap(corr_pearson, annot=True, cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Pearson Correlation')
plt.show()

fig = plt.figure(figsize = (10,8))
sns.heatmap(corr_spearman, annot=True, cmap='RdYlGn', vmin=-1, vmax=+1)
plt.title('Spearman Correlation')
plt.show()

In [ ]:
# example of scatter plot - we pick pair having highest (Pearson) correlation
sns.jointplot(data=df_train, x='cont1', y='cont2', kind='hex')
plt.show()

<a id='3'></a>
# Categorical Features

In [ ]:
features_cat = ['cat0', 'cat1', 'cat2', 'cat3',
                'cat4', 'cat5', 'cat6', 'cat7',
                'cat8', 'cat9', 'cat10', 'cat11',
                'cat12', 'cat13', 'cat14', 'cat15',
                'cat16', 'cat17', 'cat18']

In [ ]:
# plot distribution of categorical features
for f in features_cat:
    plt.figure(figsize=(14,4))
    df_train[f].value_counts().plot(kind='bar')
    plt.title(f)
    plt.grid()
    plt.show()

#### Well, "cat10" has lots of different values, this might require a closer look...

In [ ]:
# count different values/levels
cat10_freq = df_train.cat10.value_counts()
print(cat10_freq)

# and plot frequency distribution using log scale
fig, ax = plt.subplots(figsize=(12,4))
ax.plot(np.log10(cat10_freq))
ax.xaxis.set_major_locator(plt.MaxNLocator(20)) # reduce number of x-axis labels
plt.title('cat10 - Frequencies')
plt.ylabel('log10(Frequency)')
plt.grid()
plt.show()

In [ ]:
# evaluate mean of target by level
cat10_target = df_train.groupby(['cat10']).agg({
    'target' : ['mean','count']})
# ... and sort by frequency of level
cat10_target = cat10_target.sort_values([('target','count')], ascending=False)

# plot mean of target by level; bubble area ~ frequency
fig, ax = plt.subplots(figsize=(12,6))
ax.scatter(cat10_target.index, cat10_target[('target','mean')],
           s=2*np.sqrt(cat10_target[('target','count')]),
           alpha = 0.5)
ax.xaxis.set_major_locator(plt.MaxNLocator(20)) # reduce number of x-axis labels
plt.title('cat10 - Average target by level (bubble area ~ frequency)')
plt.grid()
plt.show()

#### => It could be beneficial to group the less frequent levels (e. g. right of "CP") into a group "other".

In [ ]:
# let's give it a try: define levels to be kept
n_keep = 201
cat10_keep = cat10_freq[0:n_keep].index.tolist()
print(cat10_keep)

In [ ]:
# add new column with reduced number of levels
df_train['cat10_reduced'] = df_train.cat10.where(df_train.cat10.isin(cat10_keep), '_OTHER_')
df_train.cat10_reduced.value_counts()

In [ ]:
# check frequency of _OTHER_ category
df_train[df_train.cat10_reduced=='_OTHER_'].cat10_reduced.value_counts()

In [ ]:
# same for test set!
df_test['cat10_reduced'] = df_test.cat10.where(df_test.cat10.isin(cat10_keep), '_OTHER_')
df_test.cat10_reduced.value_counts()

In [ ]:
# update feature list accordingly
features_cat = ['cat0', 'cat1', 'cat2', 'cat3',
                'cat4', 'cat5', 'cat6', 'cat7',
                'cat8', 'cat9', 'cat10_reduced', 'cat11',
                'cat12', 'cat13', 'cat14', 'cat15',
                'cat16', 'cat17', 'cat18']

<a id='4'></a>
# Target vs Features

## Numerical Features

In [ ]:
# plot target vs binned numerical features using mosaic plot
plt_para_save = plt.rcParams['figure.figsize'] # remember plot settings

for f in features_num:
    
    # add binned version of each numerical feature first
    new_var = f + '_bin'
    df_train[new_var] = pd.qcut(df_train[f], 10)
    
    # then create mosaic plot
    plt.rcParams["figure.figsize"] = (16,6) # increase plot size for mosaics
    mosaic(df_train, [new_var, 'target'], title='Target vs ' + f + ' [binned]')
    plt.show()
    
# reset plot size again
plt.rcParams['figure.figsize'] = plt_para_save

## Categorical Features

In [ ]:
# plot target vs features using mosaic plot
plt_para_save = plt.rcParams['figure.figsize'] # remember plot settings

for f in features_cat:
    plt.rcParams["figure.figsize"] = (16,6) # increase plot size for mosaics
    mosaic(df_train, [f, 'target'], title='Target vs ' + f + ' [binned]')
    plt.show()
    
# reset plot size again
plt.rcParams['figure.figsize'] = plt_para_save

<a id='5'></a>
# Build GLM Model

In [ ]:
# select predictors
predictors = features_num + features_cat
print('Number of predictors: ', len(predictors))
print(predictors)

In [ ]:
# start H2O
h2o.init(max_mem_size='12G', nthreads=4) # Use maximum of 12 GB RAM and 4 cores

In [ ]:
# upload data frames in H2O environment
t1 = time.time()
train_hex = h2o.H2OFrame(df_train)
test_hex = h2o.H2OFrame(df_test)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

# force categorical target
train_hex['target'] = train_hex['target'].asfactor()

In [ ]:
# fit GLM model
n_cv = 5
fit_1 = H2OGeneralizedLinearEstimator(nfolds=n_cv,
                                      family='binomial',
                                      alpha=0,
                                      Lambda=0,
                                      seed=999)

# train model
t1 = time.time()
fit_1.train(x=predictors,
            y='target',
            training_frame=train_hex)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

In [ ]:
# show cross validation metrics
fit_1.cross_validation_metrics_summary()

### Variable Importance

In [ ]:
# basic version
fit_1.varimp_plot(20)

## Check performance on training data / cross validations

In [ ]:
# training performance
perf_train = fit_1.model_performance(train=True)
perf_train.plot()

In [ ]:
# cross validation performance
perf_cv = fit_1.model_performance(xval=True)
perf_cv.plot()

<a id='6'></a>
# Build Random Forest Model

In [ ]:
# Random Forest model
n_cv = 5
fit_2 = H2ORandomForestEstimator(nfolds=n_cv,
                                 distribution='bernoulli',
                                 ntrees=100,
                                 mtries=-1, # automatic selection
                                 max_depth=20,
                                 score_each_iteration=True,
                                 stopping_metric='auc',
                                 stopping_rounds=5,
                                 stopping_tolerance=0.0001,
                                 seed=999)

# train model
t1 = time.time()
fit_2.train(x=predictors,
            y='target',
            training_frame=train_hex)
t2 = time.time()
print('Elapsed time [s]: ', np.round(t2-t1,2))

In [ ]:
# show cross validation metrics
fit_2.cross_validation_metrics_summary()

In [ ]:
# show scoring history - training vs cross validations
for i in range(n_cv):
    cv_model_temp = fit_2.cross_validation_models()[i]
    df_cv_score_history = cv_model_temp.score_history()
    my_title = 'CV ' + str(1+i) + ' - Scoring History [AUC]'
    plt.scatter(df_cv_score_history.number_of_trees,
                y=df_cv_score_history.training_auc, 
                c='blue', label='training')
    plt.scatter(df_cv_score_history.number_of_trees,
                y=df_cv_score_history.validation_auc, 
                c='darkorange', label='validation')
    plt.title(my_title)
    plt.xlabel('Number of Trees')
    plt.ylabel('AUC')
    plt.ylim(0.7,1)
    plt.legend()
    plt.grid()
    plt.show()


In [ ]:
# training performance
perf_train = fit_2.model_performance(train=True)
perf_train.plot()

In [ ]:
# cross validation performance
perf_cv = fit_2.model_performance(xval=True)
perf_cv.plot()

<a id='7'></a>
# Predict on Test Set and prepare submissions

In [ ]:
# predict on test set (extract probabilities only)
pred_test_GLM = fit_1.predict(test_hex)['p1']
pred_test_GLM = pred_test_GLM.as_data_frame().p1

# plot test set predictions (probabilities)
plt.figure(figsize=(6,4))
plt.hist(pred_test_GLM, bins=100)
plt.title('Predictions on Test Set - GLM')
plt.grid()
plt.show()

In [ ]:
# predict on test set (extract probabilities only)
pred_test_RF = fit_2.predict(test_hex)['p1']
pred_test_RF = pred_test_RF.as_data_frame().p1

# plot test set predictions (probabilities)
plt.figure(figsize=(6,4))
plt.hist(pred_test_RF, bins=100)
plt.title('Predictions on Test Set - RF')
plt.grid()
plt.show()

In [ ]:
# combine predictions in one data frame
df_preds_test = pd.DataFrame({'GLM': pred_test_GLM.values, 'RF': pred_test_RF.values})

In [ ]:
# scatter plot of two prediction sets
sns.jointplot(data=df_preds_test, x='GLM', y='RF',
              joint_kws={'s' : 2},
              alpha=0.1)
plt.show()

In [ ]:
# correlation
df_preds_test.corr()

In [ ]:
# GLM submission
df_sub_GLM = df_sub.copy()
df_sub_GLM.target = df_preds_test.GLM
display(df_sub_GLM.head())
# save to file
df_sub_GLM.to_csv('submission_GLM.csv', index=False)

In [ ]:
# RF submission
df_sub_RF = df_sub.copy()
df_sub_RF.target = df_preds_test.RF
display(df_sub_RF.head())
# save to file
df_sub_RF.to_csv('submission_RF.csv', index=False)

### Blend two models

In [ ]:
# blend two model results
df_sub_blend = df_sub.copy()
df_sub_blend.target = 0.5*df_preds_test.GLM + 0.5*df_preds_test.RF
display(df_sub_blend.head())
# save to file
df_sub_blend.to_csv('submission_blend.csv', index=False)